In [2]:
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np

In [3]:
dataset = load_dataset('glue', 'sst2')
metric = load_metric('glue', 'sst2')

Reusing dataset glue (/home/mbc7bu/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [4]:
tokenizer = AutoTokenizer.from_pretrained('roberta-base', use_fast = True)
def preprocessor(examples):
    return tokenizer(examples['sentence'], truncation=True)
encoded_dataset = dataset.map(preprocessor, batched=True)


Loading cached processed dataset at /home/mbc7bu/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-ccd8c19f412c0cd4.arrow
Loading cached processed dataset at /home/mbc7bu/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-8d47ef4e6297df3c.arrow
Loading cached processed dataset at /home/mbc7bu/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-1cc0bc90af367616.arrow


In [5]:
min(encoded_dataset["test"]['label'])

-1

In [6]:
model = AutoModelForSequenceClassification.from_pretrained('./roberta-finetuned-sst2/checkpoint-8420', num_labels = 2)

In [8]:
batch_size = 16
args = TrainingArguments(
    output_dir = 'roberta-finetuned-sst2',
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    per_device_train_batch_size= batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate = 2e-5,
    num_train_epochs = 1,
    weight_decay = 0.01,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
)

In [9]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis = 1)
    return metric.compute(predictions=predictions, references=labels)

In [10]:
trainer = Trainer(
    model = model,
    args = args,
    train_dataset = encoded_dataset["train"],
    eval_dataset = encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics = compute_metrics
)

In [ ]:
trainer.train()

In [12]:
def convert_test_label(examples):
    examples["label"] = (examples["label"] + 1)/2
    return examples

test_encoded_dataset = encoded_dataset['test'].map(convert_test_label)
trainer.evaluate(encoded_dataset['validation'])

Loading cached processed dataset at /home/mbc7bu/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-f728e3b285785fa9.arrow
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: idx, sentence. If idx, sentence are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 872
  Batch size = 16


{'eval_loss': 0.36294472217559814,
 'eval_accuracy': 0.9357798165137615,
 'eval_runtime': 0.9718,
 'eval_samples_per_second': 897.275,
 'eval_steps_per_second': 56.594}